# Take all content of post and comment and save to file for further analysis



In [4]:
import json
import os
from pathlib import Path

# Define data directory
DATA_DIR = Path("h:/Dev/University/DataScience/Project/data_preparation/influencer_data")
OUTPUT_DIR = Path("h:/Dev/University/DataScience/Project/data_preparation/outputs")

def get_json_list(dir_path):
    """Extract content and metadata from a post"""
    
    # take list of all json files in the directory dir_path
    post_files = [f for f in os.listdir(dir_path) if f.endswith(".json")]
    print(f"Found {len(post_files)} post files in {dir_path}")
    print("Sample files:", post_files[:3])
    return post_files

In [10]:
def collect_all_content(json_files):
    with open(DATA_DIR / json_files, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Extract all content for analysis
    extracted_content = {
        "posts": [],
        "comments": [],
        "statistics": {
            "total_posts": len(data),
            "total_comments": 0,
            "total_nested_comments": 0
        }
    }

    def extract_comment_content(comment, depth=0):
        """Recursively extract comment content"""
        comment_data = {
            "username": comment.get("username"),
            "content": comment.get("content"),
            "likes": comment.get("likes"),
            "retweet": comment.get("retweet"),
            "share": comment.get("share"),
            "views": comment.get("views"),
            "replies": comment.get("replies"),
            "link": comment.get("link"),
            "depth": depth
        }
        
        extracted_content["comments"].append(comment_data)
        
        if depth == 0:
            extracted_content["statistics"]["total_comments"] += 1
        else:
            extracted_content["statistics"]["total_nested_comments"] += 1
        
        # Process nested replies
        if "replies_content" in comment and comment["replies_content"]:
            for reply in comment["replies_content"]:
                extract_comment_content(reply, depth + 1)

    # Process each post
    for post in data:
        post_data = {
            "username": post.get("username"),
            "content": post.get("content"),
            "likes": post.get("likes"),
            "retweet": post.get("retweet"),
            "share": post.get("share"),
            "views": post.get("views"),
            "replies": post.get("replies"),
            "link": post.get("link"),
            "num_direct_comments": len(post.get("replies_content", []))
        }
        
        extracted_content["posts"].append(post_data)
        
        # Process all comments for this post
        if "replies_content" in post and post["replies_content"]:
            for comment in post["replies_content"]:
                extract_comment_content(comment, depth=0)

    # Save extracted content
    output_file = OUTPUT_DIR / json_files.replace(".json", "_extracted_content.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(extracted_content, f, indent=2, ensure_ascii=False)

    print(f"✅ Content extracted and saved to: {output_file}")
    print(f"\n📊 Statistics:")
    print(f"   - Total posts: {extracted_content['statistics']['total_posts']}")
    print(f"   - Total direct comments: {extracted_content['statistics']['total_comments']}")
    print(f"   - Total nested comments: {extracted_content['statistics']['total_nested_comments']}")
    print(f"   - Total all comments: {extracted_content['statistics']['total_comments'] + extracted_content['statistics']['total_nested_comments']}")

    # Also create a flat text file with all content for easy reading
    posts_file = OUTPUT_DIR / "post" / json_files.replace(".json", "_all_content.txt")
    comments_file = OUTPUT_DIR / "comments" / json_files.replace(".json", "_all_content.txt")
    # posts_file = OUTPUT_DIR / "all_posts_content.txt"
    # comments_file = OUTPUT_DIR / "all_comments_content.txt"

    # lists to accumulate flat lines for this file
    post_lines = []
    comment_lines = []

    # Extract all content for analysis (kept for JSON/summary outputs)
    extracted_content = {
        "posts": [],
        "comments": [],
        "statistics": {
            "total_posts": len(data),
            "total_comments": 0,
            "total_nested_comments": 0
        }
    }

    def extract_comment_content(comment, depth=0):
        """Recursively extract comment content"""
        comment_data = {
            "username": comment.get("username"),
            "content": comment.get("content"),
            "likes": comment.get("likes"),
            "retweet": comment.get("retweet"),
            "share": comment.get("share"),
            "views": comment.get("views"),
            "replies": comment.get("replies"),
            "link": comment.get("link"),
            "depth": depth
        }

        # add flat comment content (single line)
        c = (comment.get("content") or "").strip()
        if c:
            comment_lines.append(" ".join(c.splitlines()))

        extracted_content["comments"].append(comment_data)

        if depth == 0:
            extracted_content["statistics"]["total_comments"] += 1
        else:
            extracted_content["statistics"]["total_nested_comments"] += 1

        # Process nested replies
        if "replies_content" in comment and comment["replies_content"]:
            for reply in comment["replies_content"]:
                extract_comment_content(reply, depth + 1)

    # Process each post
    for post in data:
        # collect flat post content (single line)
        p = (post.get("content") or "").strip()
        if p:
            post_lines.append(" ".join(p.splitlines()))

        post_data = {
            "username": post.get("username"),
            "content": post.get("content"),
            "likes": post.get("likes"),
            "retweet": post.get("retweet"),
            "share": post.get("share"),
            "views": post.get("views"),
            "replies": post.get("replies"),
            "link": post.get("link"),
            "num_direct_comments": len(post.get("replies_content", []))
        }

        extracted_content["posts"].append(post_data)

        # Process all comments for this post
        if "replies_content" in post and post["replies_content"]:
            for comment in post["replies_content"]:
                extract_comment_content(comment, depth=0)

    # Save extracted JSON as before
    output_file = OUTPUT_DIR / json_files.replace(".json", "_extracted_content.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(extracted_content, f, indent=2, ensure_ascii=False)

    # Append flat lines to global text files (one content per line)
    with open(posts_file, "a", encoding="utf-8") as f:
        for line in post_lines:
            f.write(line + "\n")

    with open(comments_file, "a", encoding="utf-8") as f:
        for line in comment_lines:
            f.write(line + "\n")


    # Create summary statistics file
    summary_file = OUTPUT_DIR / json_files.replace(".json", "_summary.json")
    summary = {
        "total_posts": len(data),
        "posts_with_comments": sum(1 for post in data if post.get("replies_content")),
        "total_direct_comments": extracted_content['statistics']['total_comments'],
        "total_nested_comments": extracted_content['statistics']['total_nested_comments'],
        "engagement_stats": {
            "total_likes": sum(post.get("likes", 0) for post in data),
            "total_retweets": sum(post.get("retweet", 0) for post in data),
            "total_views": sum(post.get("views", 0) for post in data if post.get("views")),
            "avg_likes_per_post": sum(post.get("likes", 0) for post in data) / len(data),
            "avg_views_per_post": sum(post.get("views", 0) for post in data if post.get("views")) / len([p for p in data if p.get("views")])
        }
    }

    with open(summary_file, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)

    print(f"✅ Summary statistics saved to: {summary_file}")
    print(f"\n📈 Engagement Overview:")
    print(f"   - Total likes across all posts: {summary['engagement_stats']['total_likes']:,}")
    print(f"   - Total retweets: {summary['engagement_stats']['total_retweets']:,}")
    print(f"   - Total views: {summary['engagement_stats']['total_views']:,}")
    print(f"   - Average likes per post: {summary['engagement_stats']['avg_likes_per_post']:.0f}")
    print(f"   - Average views per post: {summary['engagement_stats']['avg_views_per_post']:,.0f}")

In [12]:
all_json_files = get_json_list(DATA_DIR)
for json_file in all_json_files:
    try:
        collect_all_content(json_file)
    except Exception as e:
        print(f"❌ Error processing {json_file}: {e}")

Found 71 post files in h:\Dev\University\DataScience\Project\data_preparation\influencer_data
Sample files: ['AdamParkhomenko.json', 'ArielleScarcell.json', 'AstroTerry.json']
✅ Content extracted and saved to: h:\Dev\University\DataScience\Project\data_preparation\outputs\AdamParkhomenko_extracted_content.json

📊 Statistics:
   - Total posts: 40
   - Total direct comments: 647
   - Total nested comments: 992
   - Total all comments: 1639
✅ Summary statistics saved to: h:\Dev\University\DataScience\Project\data_preparation\outputs\AdamParkhomenko_summary.json

📈 Engagement Overview:
   - Total likes across all posts: 216,102
   - Total retweets: 44,952
   - Total views: 6,251,398
   - Average likes per post: 5403
   - Average views per post: 156,285
✅ Content extracted and saved to: h:\Dev\University\DataScience\Project\data_preparation\outputs\ArielleScarcell_extracted_content.json

📊 Statistics:
   - Total posts: 33
   - Total direct comments: 359
   - Total nested comments: 2027
   -

# EDA

Link in a tweet is hided behind a structure https://t.co/xxxxx. We need to extract the full link to analyze what it links to

In [1]:
import requests

def expand_twitter_link(url):
    try:
        r = requests.head(url, allow_redirects=True, timeout=5)
        return r.url
    except requests.exceptions.RequestException as e:
        return None


In [4]:
link = " https://t.co/YwR0scMf5F"
print(expand_twitter_link(link))

https://twitter.com/mrlourage/status/1975311155112607859/photo/1


In [7]:
import re
import csv
from pathlib import Path
import requests
from tqdm.notebook import tqdm
import json

def extract_and_resolve_links(comments_path, posts_path, output_links):
    """
    Extract all links from text files in comments_path and posts_path directories,
    resolve shortened URLs, and save to CSV file incrementally.
    
    Args:
        comments_path: Path to directory containing comment text files
        posts_path: Path to directory containing post text files
        output_links: Path to output directory for links CSV
    """
    # Ensure output directory exists
    output_dir = Path(output_links)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Output CSV file and progress tracker
    csv_file = output_dir / "links.csv"
    progress_file = output_dir / "progress.json"
    
    # Load completed files from previous run
    completed_files = set()
    if progress_file.exists():
        with open(progress_file, "r", encoding="utf-8") as f:
            completed_files = set(json.load(f).get("completed_files", []))
        print(f"📋 Resuming from previous run. {len(completed_files)} files already processed.")
    
    # Check if CSV exists, if not create with header
    csv_exists = csv_file.exists()
    
    # Regular expression to find URLs
    url_pattern = re.compile(r'https?://[^\s]+')
    
    def process_directory(dir_path, category, csv_exists=csv_exists):
        dir_path = Path(dir_path)
        if not dir_path.exists():
            print(f"⚠️  Directory not found: {dir_path}")
            return

        txt_files = list(dir_path.glob("*.txt"))
        # Filter out already completed files
        txt_files = [f for f in txt_files if f.name not in completed_files]
        
        if not txt_files:
            print(f"✅ All files in {dir_path} ({category}) already processed!")
            return
        
        print(f"\nProcessing {len(txt_files)} files in {dir_path} ({category})")

        for txt_file in tqdm(txt_files, desc=f"Processing {category}s", unit="file", position=0):
            try:
                with open(txt_file, "r", encoding="utf-8") as f:
                    content = f.read()
                urls = url_pattern.findall(content)
                
                # Collect links for this file
                file_links = []
                for url in tqdm(urls, desc=f"  Resolving links in {txt_file.name}", leave=False, unit="link", position=1):
                    url = url.rstrip('.,;:!?)')
                    resolved_url = expand_twitter_link(url)
                    file_links.append({
                        "link": url,
                        "resolved_link": resolved_url if resolved_url else url,
                        "category": category,
                        "source_file": txt_file.name
                    })
                
                # Append to CSV immediately after processing each file
                with open(csv_file, "a", newline="", encoding="utf-8") as f:
                    fieldnames = ["link", "resolved_link", "category", "source_file"]
                    writer = csv.DictWriter(f, fieldnames=fieldnames)
                    
                    # Write header only if file is new
                    if not csv_exists:
                        writer.writeheader()
                        csv_exists = True
                    
                    writer.writerows(file_links)
                
                # Mark file as completed
                completed_files.add(txt_file.name)
                
                # Save progress after each file
                with open(progress_file, "w", encoding="utf-8") as f:
                    json.dump({"completed_files": list(completed_files)}, f, indent=2)
                
            except Exception as e:
                tqdm.write(f"❌ Error processing {txt_file}: {e}")
    
    # Process both directories
    print("\n📂 Extracting links from files...")
    # process_directory(posts_path, "posts")
    process_directory(comments_path, "comments")
    
    # Count total links in CSV
    total_links = 0
    if csv_file.exists():
        with open(csv_file, "r", encoding="utf-8") as f:
            total_links = sum(1 for line in f) - 1  # Subtract header
    
    print(f"\n✅ Links saved successfully!")
    print(f"   - Total links in CSV: {total_links}")
    print(f"   - Total files processed: {len(completed_files)}")
    print(f"   - Progress saved to: {progress_file}")
    
    return completed_files

In [8]:
comments_path = "outputs/comments"
posts_path = "outputs/post"
output_links = "outputs/links"

# Extract and resolve all links
links_data = extract_and_resolve_links(comments_path, posts_path, output_links)


📂 Extracting links from files...

Processing 71 files in outputs\comments (comments)


Processing commentss:   0%|          | 0/71 [00:00<?, ?file/s]

  Resolving links in AdamParkhomenko_all_content.txt:   0%|          | 0/366 [00:00<?, ?link/s]

  Resolving links in ArielleScarcell_all_content.txt:   0%|          | 0/348 [00:00<?, ?link/s]

  Resolving links in AstroTerry_all_content.txt:   0%|          | 0/311 [00:00<?, ?link/s]

...

  Resolving links in WBrettWilson_all_content.txt:   0%|          | 0/176 [00:00<?, ?link/s]

  Resolving links in WhiteHouse_all_content.txt:   0%|          | 0/2148 [00:00<?, ?link/s]

  Resolving links in yarahawari_all_content.txt:   0%|          | 0/176 [00:00<?, ?link/s]

  Resolving links in ylecun_all_content.txt:   0%|          | 0/241 [00:00<?, ?link/s]


✅ Links saved successfully!
   - Total links in CSV: 25548
   - Total files processed: 71
   - Progress saved to: outputs\links\progress.json


In [ ]:
output_links = "outputs\links"
# In all files txt in the comments_path and posts_path directories, we get all links starting with "http" and save them to file links.txt
# save it as csv, one column "link", one column "resolved_link", one column "category"

# EDA to process tweet redundant tags

In [ ]:
# import json
# import re
# from pathlib import Path
# import emoji


# def clean_and_extract_content(json_file_path, output_dir):
#     json_file_path = Path(json_file_path)
#     output_dir = Path(output_dir)
#     output_dir.mkdir(parents=True, exist_ok=True)
    
#     # Load JSON data
#     with open(json_file_path, "r", encoding="utf-8") as f:
#         data = json.load(f)
    
#     # Get username from filename
#     author_username = json_file_path.stem
    
#     # Output files
#     posts_file = output_dir / f"{author_username}_posts_cleaned.txt"
#     comments_file = output_dir / f"{author_username}_comments_cleaned.txt"
    
#     def remove_tags(content, author=None, parent_author=None):
#         """
#         Remove @mentions at the beginning of content.
#         Removes author's own tag and parent comment author tag.
#         """
#         if not content:
#             return ""
        
#         # Remove the author's own mention
#         if author:
#             content = re.sub(rf'@{re.escape(author)}\s*', '', content, flags=re.IGNORECASE)
        
#         # Remove parent author mention (for replies)
#         if parent_author:
#             content = re.sub(rf'@{re.escape(parent_author)}\s*', '', content, flags=re.IGNORECASE)
        
#         # Remove any leading @mentions (general cleanup)
#         content = re.sub(r'^(@\w+\s*)+', '', content)
        
#         return content.strip()
    
#     post_lines = []
#     comment_lines = []

#     def emoji_to_text(content):
#         if not content:
#             return ""
        
#         # Convert emoji to text with custom delimiters
#         demojized = emoji.demojize(content, delimiters=(" emo-", "- "))
        
#         # Replace underscores with spaces in emoji names
#         demojized = demojized.replace("_", " ")
        
#         return demojized
    
#     def hashtag_process(content):
#         if not content:
#             return ""
        
#         def split_hashtag(match):
#             tag = match.group(1)
#             # Add space before capital letters and numbers
#             spaced = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', tag)
#             spaced = re.sub(r'([a-z])([A-Z])', r'\1 \2', spaced)
#             return f"hashtag-{spaced}-"
        
#         content = re.sub(r'#(\w+)', split_hashtag, content)
#         return content
    
#     def extract_comment_content(comment, parent_author=None, depth=0):
#         """Recursively extract and clean comment content"""
#         username = comment.get("username")
#         content = comment.get("content", "")
        
#         # Clean the content
#         no_tag_content = remove_tags(content, author=username, parent_author=parent_author)
#         no_emoji_content = emoji_to_text(no_tag_content)
#         no_hashtag_content = hashtag_process(no_emoji_content)

#         cleaned_content = no_hashtag_content.strip()
#         # Add to list if not empty
#         if cleaned_content:
#             # Collapse newlines to single line
#             single_line = " ".join(cleaned_content.splitlines())
#             comment_lines.append(single_line)
        
#         # Process nested replies
#         if "replies_content" in comment and comment["replies_content"]:
#             for reply in comment["replies_content"]:
#                 extract_comment_content(reply, parent_author=username, depth=depth + 1)
    
#     # Process each post
#     for post in data:
#         username = post.get("username")
#         content = post.get("content", "")
        
#         # Clean post content (remove author's own tag)
#         cleaned_content = remove_tags(content, author=username)
        
#         # Add to post lines if not empty
#         if cleaned_content:
#             single_line = " ".join(cleaned_content.splitlines())
#             post_lines.append(single_line)
        
#         # Process all comments for this post
#         if "replies_content" in post and post["replies_content"]:
#             for comment in post["replies_content"]:
#                 # Pass post author as parent for top-level comments
#                 extract_comment_content(comment, parent_author=username, depth=0)
    
#     # Save cleaned posts
#     with open(posts_file, "w", encoding="utf-8") as f:
#         for line in post_lines:
#             f.write(line + "\n")
    
#     # Save cleaned comments
#     with open(comments_file, "w", encoding="utf-8") as f:
#         for line in comment_lines:
#             f.write(line + "\n")
    
#     print(f"✅ Cleaned content saved:")
#     print(f"   - Posts: {posts_file} ({len(post_lines)} lines)")
#     print(f"   - Comments: {comments_file} ({len(comment_lines)} lines)")
    
#     return {
#         "posts_file": posts_file,
#         "comments_file": comments_file,
#         "posts_count": len(post_lines),
#         "comments_count": len(comment_lines)
#     }

# # Process all JSON files in a directory
# def process_all_influencer_files(input_dir, output_dir):
#     """
#     Process all JSON files in input directory and save cleaned content.
    
#     Args:
#         input_dir: Directory containing JSON files
#         output_dir: Directory to save cleaned content
#     """
#     input_dir = Path(input_dir)
#     json_files = list(input_dir.glob("*.json"))
    
#     print(f"Found {len(json_files)} JSON files to process\n")
    
#     results = []
#     for json_file in json_files:
#         print(f"\n📄 Processing: {json_file.name}")
#         try:
#             result = clean_and_extract_content(json_file, output_dir)
#             results.append(result)
#         except Exception as e:
#             print(f"❌ Error processing {json_file.name}: {e}")
    
#     # Summary
#     total_posts = sum(r["posts_count"] for r in results)
#     total_comments = sum(r["comments_count"] for r in results)
    
#     print(f"\n{'='*60}")
#     print(f"📊 Processing Complete!")
#     print(f"   - Files processed: {len(results)}/{len(json_files)}")
#     print(f"   - Total posts: {total_posts}")
#     print(f"   - Total comments: {total_comments}")
#     print(f"   - Output directory: {output_dir}")
#     print(f"{'='*60}")
    
#     return results

In [ ]:
import json
import re
from pathlib import Path
import csv
import emoji
from datetime import datetime

def clean_and_extract_content_advanced(json_file_path, output_dir, links_csv_path):
    """
    Advanced cleaning of content with comprehensive text processing.
    
    Args:
        json_file_path: Path to input JSON file
        output_dir: Directory to save cleaned content
        links_csv_path: Path to links CSV file for link resolution
    """
    json_file_path = Path(json_file_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Load JSON data
    with open(json_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Get username from filename
    author_username = json_file_path.stem
    
    # Output files
    posts_file = output_dir / f"{author_username}_posts_cleaned.txt"
    comments_file = output_dir / f"{author_username}_comments_cleaned.txt"
    corner_cases_file = output_dir / f"{author_username}_corner_cases.txt"
    
    # Load links mapping from CSV
    links_map = {}
    if Path(links_csv_path).exists():
        with open(links_csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                links_map[row["link"]] = row["resolved_link"]
    
    post_lines = []
    comment_lines = []
    corner_cases = []
    
    # Internet slang/shorthand mapping
    slang_map = {
        "lol": "laugh out loud",
        "lmao": "laughing",
        "lmfao": "laughing",
        "haha": "laughing",
        "lmk": "let me know",
        "rofl": "laughing",
        "omg": "oh my god",
        "wtf": "what the fuck",
        "btw": "by the way",
        "imo": "in my opinion",
        "imho": "in my humble opinion",
        "tbh": "to be honest",
        "smh": "shaking my head",
        "fyi": "for your information",
        "idk": "i do not know",
        "irl": "in real life",
        "brb": "be right back",
        "af": "as fuck",
        "rn": "right now",
        "dm": "direct message",
        "rt": "retweet",
        "ngl": "not gonna lie",
        "fr": "for real",
        "bc": "because",
        "tho": "though",
        "thru": "through",
        "ppl": "people",
        "plz": "please",
        "thx": "thanks",
        "u": "you",
        "ur": "your",
        "y": "why",
        "r": "are",
        "b4": "before",
        "gr8": "great",
        "m8": "mate",
        "2day": "today",
        "2nite": "tonight",
        "tmr": "tomorrow",
        "tmrw": "tomorrow",
        "asap": "as soon as possible",
        "aka": "also known as",
        "fomo": "fear of missing out",
        "goat": "greatest of all time",
        "sus": "suspicious",
        "lowkey": "somewhat",
        "highkey": "very",
        "gtfo": "get the fuck out",
        "stfu": "shut the fuck up",
        "jk": "just kidding",
        "ikr": "i know right",
        "tfw": "that feeling when",
        "mfw": "my face when",
        "yolo": "you only live once",
        "fam": "family",
        "bro": "brother",
        "sis": "sister",
        "gonna": "going to",
        "wanna": "want to",
        "gotta": "got to",
        "kinda": "kind of",
        "sorta": "sort of",
        "dunno": "do not know",
        "cuz": "because",
        "w/": "with",
        "w/o": "without",
        "pov": "point of view",
        "rip": "rest in peace",
        "ftw": "for the win",
        "gg": "good game",
        "wp": "well played",
        "ez": "easy",
        "op": "overpowered",
        "af": "as fuck",
        "lit": "amazing",
        "salty": "upset",
        "savage": "brutal",
        "flex": "show off",
        "stan": "support",
        "simp": "obsessed",
        "based": "authentic",
        "ratio": "more engagement",
        "copium": "denial",
        "hopium": "hope",
        "oof": "expression of pain",
        "yeet": "throw",
        "bet": "okay",
        "cap": "lie",
        "no cap": "no lie",
        "bussin": "really good",
        "slaps": "really good",
        "hits different": "unique experience",
        "vibe": "feeling",
        "mood": "relatable",
        "same": "i agree",
        "facts": "truth",
        "periodt": "end of discussion",
        "tea": "gossip",
        "spill the tea": "share gossip",
        "shade": "disrespect",
        "throw shade": "insult",
        "slay": "do great",
        "queen": "amazing woman",
        "king": "amazing man",
        "w": "win",
        "l": "loss"
    }
    
    # Common words to remove (stop words and contractions)
    remove_words = {
        "'s", "'re", "'ll", "'ve", "'d", "'m",
        "the", "a", "an", "and", "or", "but", "in", "on", "at", "to", "for",
        "of", "with", "by", "from", "as", "is", "was", "are", "were", "be",
        "been", "being", "have", "has", "had", "do", "does", "did",
        "will", "would", "should", "could", "may", "might", "must",
        "can", "shall", "this", "that", "these", "those"
    }
    
    # Negative terms to replace with "not"
    negative_terms = {
        "not", "dont", "don't", "wont", "won't", "isnt", "isn't",
        "arent", "aren't", "wasnt", "wasn't", "werent", "weren't",
        "hasnt", "hasn't", "havent", "haven't", "hadnt", "hadn't",
        "doesnt", "doesn't", "didnt", "didn't", "wouldnt", "wouldn't",
        "shouldnt", "shouldn't", "couldnt", "couldn't", "cant", "can't",
        "mustnt", "mustn't", "no", "never", "neither", "nobody", "nothing", "nowhere"
    }
    
    def normalize_repeated_chars(text):
        """
        Normalize repeated characters to max 2 occurrences.
        Examples: wahhhhhh -> wahh, gooooood -> good
        """
        # Replace 3+ repeated characters with max 2
        return re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    def is_media_link(url):
        """Check if URL is media (photo/video)"""
        media_indicators = ["/photo/", "/video/", "/status/", "twimg.com", "pbs.twimg.com"]
        return any(indicator in url.lower() for indicator in media_indicators)
    
    def classify_number(text, number):
        """Classify if number is year/date/time or generic number"""
        try:
            # Year pattern (1900-2099)
            if 1900 <= int(number) <= 2099:
                return "year"
        except ValueError:
            pass
        
        # Date patterns (MM/DD, DD-MM, etc.)
        if re.search(r'\d{1,2}[/-]\d{1,2}', text):
            return "date"
        
        # Time patterns (HH:MM)
        if re.search(r'\d{1,2}:\d{2}', text):
            return "time"
        
        return "number"
    
    def clean_text(content, author=None, parent_author=None):
        """
        Comprehensive text cleaning pipeline.
        """
        if not content:
            return "", []
        
        original_content = content
        has_non_ascii = False
        non_ascii_words = []
        
        # Step 1: Remove RT and following @mention
        content = re.sub(r'^RT\s+@\w+:\s*', '', content, flags=re.IGNORECASE)
        
        # Step 2: Remove author's own mention
        if author:
            content = re.sub(rf'@{re.escape(author)}\s*', '', content, flags=re.IGNORECASE)
        
        # Step 3: Remove parent author mention (for replies)
        if parent_author:
            content = re.sub(rf'@{re.escape(parent_author)}\s*', '', content, flags=re.IGNORECASE)
        
        # Step 4: Remove leading @mentions
        content = re.sub(r'^(@\w+\s*)+', '', content)
        
        # Step 5: Replace remaining @mentions with "tag"
        content = re.sub(r'@\w+', 'tag', content)
        
        # Step 6: Process links
        def replace_link(match):
            url = match.group(0).rstrip('.,;:!?)')
            resolved_url = links_map.get(url, url)
            
            if is_media_link(resolved_url):
                return "media"
            else:
                return "link"
        
        content = re.sub(r'https?://[^\s]+', replace_link, content)
        
        # Step 7: Process hashtags - remove # and add spaces
        def process_hashtag(match):
            tag = match.group(1)
            # Add space before capital letters and numbers
            spaced = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', tag)
            spaced = re.sub(r'([a-z])([A-Z])', r'\1 \2', spaced)
            return spaced.lower()
        
        content = re.sub(r'#(\w+)', process_hashtag, content)
        
        # Step 8: Replace emojis with "emoji"
        content = emoji.replace_emoji(content, replace='')
        
        # Step 9: Replace numbers with suffixes (7k, 213m, 5b, etc.) with "number"
        content = re.sub(r'\b\d+\.?\d*[kmbtKMBT]\b', 'number', content)
        
        # Step 10: Process regular numbers
        def replace_number(match):
            num = match.group(0)
            num_type = classify_number(content, num)
            return num_type
        
        content = re.sub(r'\b\d+\b', replace_number, content)
        
        # Step 11: Normalize repeated characters
        content = normalize_repeated_chars(content)
        
        # Step 12: Translate internet slang/shorthand
        words = content.split()
        translated_words = []
        
        for word in words:
            word_lower = word.lower()
            # Remove punctuation for slang matching
            word_clean = re.sub(r'[^\w\s-]', '', word_lower)
            
            # Check if it's slang
            if word_clean in slang_map:
                translated_words.append(slang_map[word_clean])
            else:
                translated_words.append(word)
        
        content = ' '.join(translated_words)
        
        # Step 13: Handle contractions and negative terms
        words = content.split()
        cleaned_words = []
        
        for word in words:
            word_lower = word.lower()
            
            # Check for non-ASCII characters
            if not all(ord(c) < 128 for c in word):
                has_non_ascii = True
                non_ascii_words.append(word)
                continue  # Skip non-ASCII words
            
            # Replace negative terms with "not"
            if word_lower in negative_terms:
                cleaned_words.append("not")
            # Remove common stop words
            elif word_lower not in remove_words:
                # Remove special characters except hyphens in middle of words
                cleaned_word = re.sub(r'[^\w\s-]', '', word_lower)
                if cleaned_word and cleaned_word not in remove_words:
                    cleaned_words.append(cleaned_word)
        
        # Step 14: Join and normalize spaces
        content = ' '.join(cleaned_words)
        content = re.sub(r'\s+', ' ', content).strip()
        
        # Convert to lowercase
        content = content.lower()
        
        # Track corner cases
        corner_case_info = []
        if has_non_ascii:
            corner_case_info.append({
                "original": original_content,
                "non_ascii_words": non_ascii_words,
                "cleaned": content
            })
        
        return content, corner_case_info
    
    def extract_comment_content(comment, parent_author=None, depth=0):
        """Recursively extract and clean comment content"""
        username = comment.get("username")
        content = comment.get("content", "")
        
        # Clean the content
        cleaned_content, cases = clean_text(content, author=username, parent_author=parent_author)
        
        # Track corner cases
        corner_cases.extend(cases)
        
        # Add to list if not empty
        if cleaned_content:
            comment_lines.append(cleaned_content)
        
        # Process nested replies
        if "replies_content" in comment and comment["replies_content"]:
            for reply in comment["replies_content"]:
                extract_comment_content(reply, parent_author=username, depth=depth + 1)
    
    # Process each post
    for post in data:
        username = post.get("username")
        content = post.get("content", "")
        
        # Clean post content
        cleaned_content, cases = clean_text(content, author=username)
        
        # Track corner cases
        corner_cases.extend(cases)
        
        # Add to post lines if not empty
        if cleaned_content:
            post_lines.append(cleaned_content)
        
        # Process all comments for this post
        if "replies_content" in post and post["replies_content"]:
            for comment in post["replies_content"]:
                extract_comment_content(comment, parent_author=username, depth=0)
    
    # Save cleaned posts
    with open(posts_file, "w", encoding="utf-8") as f:
        for line in post_lines:
            f.write(line + "\n")
    
    # Save cleaned comments
    with open(comments_file, "w", encoding="utf-8") as f:
        for line in comment_lines:
            f.write(line + "\n")
    
    # Save corner cases
    if corner_cases:
        with open(corner_cases_file, "w", encoding="utf-8") as f:
            for case in corner_cases:
                f.write(f"Original: {case['original']}\n")
                f.write(f"Non-ASCII words: {', '.join(case['non_ascii_words'])}\n")
                f.write(f"Cleaned: {case['cleaned']}\n")
                f.write("-" * 80 + "\n")
    
    print(f"✅ Cleaned content saved:")
    print(f"   - Posts: {posts_file} ({len(post_lines)} lines)")
    print(f"   - Comments: {comments_file} ({len(comment_lines)} lines)")
    print(f"   - Corner cases: {corner_cases_file} ({len(corner_cases)} cases)")
    
    return {
        "posts_file": posts_file,
        "comments_file": comments_file,
        "posts_count": len(post_lines),
        "comments_count": len(comment_lines),
        "corner_cases_count": len(corner_cases)
    }

# Process all JSON files in a directory
def process_all_influencer_files_advanced(input_dir, output_dir, links_csv_path):
    """
    Process all JSON files with advanced cleaning.
    """
    input_dir = Path(input_dir)
    json_files = list(input_dir.glob("*.json"))
    
    print(f"Found {len(json_files)} JSON files to process\n")
    
    results = []
    for json_file in json_files:
        print(f"\n📄 Processing: {json_file.name}")
        try:
            result = clean_and_extract_content_advanced(json_file, output_dir, links_csv_path)
            results.append(result)
        except Exception as e:
            print(f"❌ Error processing {json_file.name}: {e}")
    
    # Summary
    total_posts = sum(r["posts_count"] for r in results)
    total_comments = sum(r["comments_count"] for r in results)
    total_corner_cases = sum(r["corner_cases_count"] for r in results)
    
    print(f"\n{'='*60}")
    print(f"📊 Processing Complete!")
    print(f"   - Files processed: {len(results)}/{len(json_files)}")
    print(f"   - Total posts: {total_posts}")
    print(f"   - Total comments: {total_comments}")
    print(f"   - Total corner cases: {total_corner_cases}")
    print(f"   - Output directory: {output_dir}")
    print(f"{'='*60}")
    
    return results

In [54]:
EXPR_DIR = "influencer_data/AdamParkhomenko.json"
OUTPUT_DIR = "outputs/cleaned_content_advanced"
LINKS_CSV_PATH = "outputs/links/links.csv"

# Process single file
result = clean_and_extract_content_advanced(
    EXPR_DIR, 
    OUTPUT_DIR, 
    LINKS_CSV_PATH
)

✅ Cleaned content saved:
   - Posts: outputs\cleaned_content_advanced\AdamParkhomenko_posts_cleaned.txt (40 lines)
   - Comments: outputs\cleaned_content_advanced\AdamParkhomenko_comments_cleaned.txt (1637 lines)
   - Corner cases: outputs\cleaned_content_advanced\AdamParkhomenko_corner_cases.txt (320 cases)


In [20]:
def emoji_to_text(content):
    if not content:
        return ""
    
    # Convert emoji to text with custom delimiters
    demojized = emoji.demojize(content, delimiters=(" emo-", "- "))
    
    # Replace underscores with spaces in emoji names
    demojized = demojized.replace("_", " ")
    
    return demojized

In [16]:
emoji_to_text("Lol, you kept asking, trying to get it to prove you right. But, you're not, so it never gave what you were looking for. Have you learned anything from this? Embarrassing 😂😂😂")

"Lol, you kept asking, trying to get it to prove you right. But, you're not, so it never gave what you were looking for. Have you learned anything from this? Embarrassing  emo-face with tears of joy-  emo-face with tears of joy-  emo-face with tears of joy- "

In [9]:
import json
import re
from pathlib import Path
import csv
import emoji
from datetime import datetime
from unidecode import unidecode

def clean_and_extract_content_advanced(json_file_path, output_dir, links_csv_path):
    """
    Advanced cleaning of content with comprehensive text processing.
    
    Args:
        json_file_path: Path to input JSON file
        output_dir: Directory to save cleaned content
        links_csv_path: Path to links CSV file for link resolution
    """
    json_file_path = Path(json_file_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Load JSON data
    with open(json_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Get username from filename
    author_username = json_file_path.stem
    
    # Output files
    posts_file = output_dir / f"{author_username}_posts_cleaned.txt"
    comments_file = output_dir / f"{author_username}_comments_cleaned.txt"
    corner_cases_file = output_dir / "corner_cases.txt"
    
    # Load links mapping from CSV
    links_map = {}
    if Path(links_csv_path).exists():
        with open(links_csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                links_map[row["link"]] = row["resolved_link"]
    
    post_lines = []
    comment_lines = []
    corner_cases = []
    
    # US State abbreviations mapping
    us_states = {
        "AL": "alabama", "AK": "alaska", "AZ": "arizona", "AR": "arkansas",
        "CA": "california", "CO": "colorado", "CT": "connecticut", "DE": "delaware",
        "FL": "florida", "GA": "georgia", "HI": "hawaii", "ID": "idaho",
        "IL": "illinois", "IN": "indiana", "IA": "iowa", "KS": "kansas",
        "KY": "kentucky", "LA": "louisiana", "ME": "maine", "MD": "maryland",
        "MA": "massachusetts", "MI": "michigan", "MN": "minnesota", "MS": "mississippi",
        "MO": "missouri", "MT": "montana", "NE": "nebraska", "NV": "nevada",
        "NH": "new hampshire", "NJ": "new jersey", "NM": "new mexico", "NY": "new york",
        "NC": "north carolina", "ND": "north dakota", "OH": "ohio", "OK": "oklahoma",
        "OR": "oregon", "PA": "pennsylvania", "RI": "rhode island", "SC": "south carolina",
        "SD": "south dakota", "TN": "tennessee", "TX": "texas", "UT": "utah",
        "VT": "vermont", "VA": "virginia", "WA": "washington", "WV": "west virginia",
        "WI": "wisconsin", "WY": "wyoming", "DC": "washington dc"
    }
    
    # Country code mapping (ISO 3166-1 alpha-2 and alpha-3)
    countries = {
        "US": "united states", "USA": "united states",
        "UK": "united kingdom", "GB": "united kingdom", "GBR": "united kingdom",
        "CA": "canada", "CAN": "canada",
        "CN": "china", "CHN": "china",
        "JP": "japan", "JPN": "japan",
        "DE": "germany", "DEU": "germany",
        "FR": "france", "FRA": "france",
        "IT": "italy", "ITA": "italy",
        "ES": "spain", "ESP": "spain",
        "AU": "australia", "AUS": "australia",
        "BR": "brazil", "BRA": "brazil",
        "IN": "india", "IND": "india",
        "RU": "russia", "RUS": "russia",
        "MX": "mexico", "MEX": "mexico",
        "KR": "south korea", "KOR": "south korea",
        "AR": "argentina", "ARG": "argentina",
        "ZA": "south africa", "ZAF": "south africa",
        "SA": "saudi arabia", "SAU": "saudi arabia",
        "TR": "turkey", "TUR": "turkey",
        "PL": "poland", "POL": "poland",
        "NL": "netherlands", "NLD": "netherlands",
        "BE": "belgium", "BEL": "belgium",
        "SE": "sweden", "SWE": "sweden",
        "NO": "norway", "NOR": "norway",
        "DK": "denmark", "DNK": "denmark",
        "FI": "finland", "FIN": "finland",
        "IE": "ireland", "IRL": "ireland",
        "PT": "portugal", "PRT": "portugal",
        "GR": "greece", "GRC": "greece",
        "CH": "switzerland", "CHE": "switzerland",
        "AT": "austria", "AUT": "austria",
        "NZ": "new zealand", "NZL": "new zealand",
        "SG": "singapore", "SGP": "singapore",
        "HK": "hong kong", "HKG": "hong kong",
        "IL": "israel", "ISR": "israel",
        "EG": "egypt", "EGY": "egypt",
        "NG": "nigeria", "NGA": "nigeria",
        "KE": "kenya", "KEN": "kenya",
        "PH": "philippines", "PHL": "philippines",
        "TH": "thailand", "THA": "thailand",
        "VN": "vietnam", "VNM": "vietnam",
        "MY": "malaysia", "MYS": "malaysia",
        "ID": "indonesia", "IDN": "indonesia",
        "PK": "pakistan", "PAK": "pakistan",
        "BD": "bangladesh", "BGD": "bangladesh",
        "UA": "ukraine", "UKR": "ukraine"
    }
    
    # Internet slang/shorthand mapping
    slang_map = {
        "lol": "laugh out loud",
        "lmao": "laughing",
        "lmfao": "laughing",
        "haha": "laughing",
        "lmk": "let me know",
        "rofl": "laughing",
        "omg": "oh my god",
        "wtf": "what the fuck",
        "btw": "by the way",
        "imo": "in my opinion",
        "imho": "in my humble opinion",
        "tbh": "to be honest",
        "smh": "shaking my head",
        "fyi": "for your information",
        "idk": "i do not know",
        "irl": "in real life",
        "brb": "be right back",
        "af": "as fuck",
        "rn": "right now",
        "dm": "direct message",
        "rt": "retweet",
        "ngl": "not gonna lie",
        "fr": "for real",
        "bc": "because",
        "tho": "though",
        "thru": "through",
        "ppl": "people",
        "plz": "please",
        "thx": "thanks",
        "u": "you",
        "ur": "your",
        "y": "why",
        "r": "are",
        "b4": "before",
        "gr8": "great",
        "m8": "mate",
        "2day": "today",
        "2nite": "tonight",
        "tmr": "tomorrow",
        "tmrw": "tomorrow",
        "asap": "as soon as possible",
        "aka": "also known as",
        "fomo": "fear of missing out",
        "goat": "greatest of all time",
        "sus": "suspicious",
        "lowkey": "somewhat",
        "highkey": "very",
        "gtfo": "get the fuck out",
        "stfu": "shut the fuck up",
        "jk": "just kidding",
        "ikr": "i know right",
        "tfw": "that feeling when",
        "mfw": "my face when",
        "yolo": "you only live once",
        "fam": "family",
        "bro": "brother",
        "sis": "sister",
        "gonna": "going to",
        "wanna": "want to",
        "gotta": "got to",
        "kinda": "kind of",
        "sorta": "sort of",
        "dunno": "do not know",
        "cuz": "because",
        "w/": "with",
        "w/o": "without",
        "pov": "point of view",
        "rip": "rest in peace",
        "ftw": "for the win",
        "gg": "good game",
        "wp": "well played",
        "ez": "easy",
        "op": "overpowered",
        "af": "as fuck",
        "lit": "amazing",
        "salty": "upset",
        "savage": "brutal",
        "flex": "show off",
        "stan": "support",
        "simp": "obsessed",
        "based": "authentic",
        "ratio": "more engagement",
        "copium": "denial",
        "hopium": "hope",
        "oof": "expression of pain",
        "yeet": "throw",
        "bet": "okay",
        "cap": "lie",
        "no cap": "no lie",
        "bussin": "really good",
        "slaps": "really good",
        "hits different": "unique experience",
        "vibe": "feeling",
        "mood": "relatable",
        "same": "i agree",
        "facts": "truth",
        "periodt": "end of discussion",
        "tea": "gossip",
        "spill the tea": "share gossip",
        "shade": "disrespect",
        "throw shade": "insult",
        "slay": "do great",
        "queen": "amazing woman",
        "king": "amazing man",
        "w": "win",
        "l": "loss"
    }

    # Negative contractions -> subject + not
    contractions_negative = {
        "don't": "do not", "doesn't": "does not", "didn't": "did not",
        "won't": "will not", "wouldn't": "would not", "shouldn't": "should not",
        "couldn't": "could not", "can't": "can not", "cannot": "can not",
        "isn't": "is not", "aren't": "are not", "wasn't": "was not", "weren't": "were not",
        "hasn't": "has not", "haven't": "have not", "hadn't": "had not",
        "mustn't": "must not", "mightn't": "might not", "needn't": "need not",
        "shan't": "shall not", "daren't": "dare not"
    }
    
    # Positive contractions - remove auxiliary/modal verbs
    contractions_positive = {
        "i'm": "i", "you're": "you", "he's": "he", "she's": "she", 
        "it's": "it", "we're": "we", "they're": "they",
        "i'll": "i", "you'll": "you", "he'll": "he", "she'll": "she",
        "it'll": "it", "we'll": "we", "they'll": "they",
        "i've": "i", "you've": "you", "we've": "we", "they've": "they",
        "i'd": "i", "you'd": "you", "he'd": "he", "she'd": "she",
        "it'd": "it", "we'd": "we", "they'd": "they"
    }
    
    # Common words to remove (stop words and contractions)
    remove_words = {
        "'s", "'re", "'ll", "'ve", "'d", "'m",
        "the", "a", "an", "and", "or", "but", "in", "on", "at", "to", "for",
        "of", "with", "by", "from", "as", "is", "was", "are", "were", "be",
        "been", "being", "have", "has", "had", "do", "does", "did",
        "will", "would", "should", "could", "may", "might", "must",
        "can", "shall", "this", "that", "these", "those"
    }
    
    # Negative terms to replace with "not"
    negative_terms = {
        "not", "dont", "don't", "wont", "won't", "isnt", "isn't",
        "arent", "aren't", "wasnt", "wasn't", "werent", "weren't",
        "hasnt", "hasn't", "havent", "haven't", "hadnt", "hadn't",
        "doesnt", "doesn't", "didnt", "didn't", "wouldnt", "wouldn't",
        "shouldnt", "shouldn't", "couldnt", "couldn't", "cant", "can't",
        "mustnt", "mustn't", "no", "never", "neither", "nobody", "nothing", "nowhere"
    }

    # Text-based emoji/kaomoji dictionary (for future use)
    text_emoji_map = {
        r"¯\\_\(ツ\)_/¯": "shrug",
        r"\(ツ\)": "happy face",
        r"ಠ_ಠ": "disapproval",
        r"ಠ益ಠ": "angry",
        r"ʘ‿ʘ": "innocent",
        r"\(╯°□°\)╯︵ ┻━┻": "table flip",
        r"┬─┬ノ\( º _ ºノ\)": "table respect",
        r"ᕕ\( ᐛ \)ᕗ": "excited",
        r"\(^_^\)": "happy",
        r"\^_\^": "happy",
        r"＼\(^o^\)／": "celebrating",
        r"٩\(◕‿◕｡\)۶": "excited",
        r"\(づ｡◕‿‿◕｡\)づ": "hug",
        r"ლ\(ಠ益ಠლ\)": "why",
        r"¯\\_\(シ\)_/¯": "shrug",
        r"\(⌐■_■\)": "cool",
        r"\(╥_╥\)": "crying",
        r"\(T_T\)": "crying",
        r"\._.": "sad",
        r";_;": "crying",
        r"T_T": "crying",
        r"o_o": "surprised",
        r"O_O": "shocked",
        r">_<": "frustrated",
        r"^_^": "happy",
        r"XD": "laughing hard",
        r"xD": "laughing",
        r":D": "big smile",
        r"=D": "big smile",
        r":]": "smile",
        r":[": "sad",
        r":(": "sad",
        r":)": "smile",
        r";)": "wink",
        r";D": "wink laugh",
    }

    # Special Unicode symbols to remove (when standalone)
    standalone_symbols = {
        "•",  # Bullet point
        "◦",  # White bullet
        "▪",  # Black small square
        "▫",  # White small square
        "·",  # Middle dot
        "※",  # Reference mark
        "→",  # Right arrow
        "←",  # Left arrow
        "↑",  # Up arrow
        "↓",  # Down arrow
        "✓",  # Check mark
        "✗",  # X mark
        "★",  # Star
        "☆",  # White star
        "♦",  # Diamond
        "♥",  # Heart
        "♠",  # Spade
        "♣",  # Club
    }
    
    def normalize_repeated_chars(text):
        """
        Normalize repeated characters to max 2 occurrences.
        Examples: wahhhhhh -> wahh, gooooood -> good
        """
        # Replace 3+ repeated characters with max 2
        return re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    def is_media_link(url):
        """Check if URL is media (photo/video)"""
        media_indicators = ["/photo/", "/video/", "/status/", "twimg.com", "pbs.twimg.com"]
        return any(indicator in url.lower() for indicator in media_indicators)
    
    def classify_number(text, number):
        """Classify if number is year/date/time or generic number"""
        try:
            # Year pattern (1900-2099)
            if 1900 <= int(number) <= 2099:
                return "year"
        except ValueError:
            pass
        
        # Date patterns (MM/DD, DD-MM, etc.)
        if re.search(r'\d{1,2}[/-]\d{1,2}', text):
            return "date"
        
        # Time patterns (HH:MM)
        if re.search(r'\d{1,2}:\d{2}', text):
            return "time"
        
        return "number"
    
    def clean_text(content, author=None, parent_author=None):
        """
        Comprehensive text cleaning pipeline.
        """
        if not content:
            return "", []
        
        original_content = content
        has_non_ascii = False
        non_ascii_words = []
        
        # Step 0: Convert Unicode characters to closest ASCII equivalents using unidecode
        # This handles accented characters like está → esta, política → politica
        content = unidecode(content)
        
        # Step 0.5: Remove specific characters from words (keep the rest of the word)
        # Remove these characters wherever they appear
        chars_to_strip = ['(', ')', '[', ']', '{', '}', '!', '?', '|', '*', '~', '`', '^', '<', '>', '¡', '•']
        for char in chars_to_strip:
            content = content.replace(char, '')
        
        # Remove leading/trailing special characters from each word (-, +, =, _, etc.)
        # This preserves hyphens in the middle of words like "well-known"
        content = re.sub(r'\b[-+=_]+', '', content)  # Remove at start of word
        content = re.sub(r'[-+=_]+\b', '', content)  # Remove at end of word
        
        # Step 1: Remove RT and following @mention
        content = re.sub(r'^RT\s+@\w+:\s*', '', content, flags=re.IGNORECASE)
        
        # Step 2: Remove author's own mention
        if author:
            content = re.sub(rf'@{re.escape(author)}\s*', '', content, flags=re.IGNORECASE)
        
        # Step 3: Remove parent author mention (for replies)
        if parent_author:
            content = re.sub(rf'@{re.escape(parent_author)}\s*', '', content, flags=re.IGNORECASE)
        
        # Step 4: Remove leading @mentions
        content = re.sub(r'^(@\w+\s*)+', '', content)
        
        # Step 5: Replace remaining @mentions with "tag"
        content = re.sub(r'@\w+', 'tag', content)
        
        # Step 6: Process links
        def replace_link(match):
            url = match.group(0).rstrip('.,;:!?)')
            resolved_url = links_map.get(url, url)
            
            if is_media_link(resolved_url):
                return "media"
            else:
                return "link"
        
        content = re.sub(r'https?://[^\s]+', replace_link, content)
        
        # Step 7: Process hashtags - remove # and add spaces
        def process_hashtag(match):
            tag = match.group(1)
            # Add space before capital letters and numbers
            spaced = re.sub(r'([a-zA-Z])([0-9])', r'\1 \2', tag)
            spaced = re.sub(r'([a-z])([A-Z])', r'\1 \2', spaced)
            return spaced.lower()
        
        content = re.sub(r'#(\w+)', process_hashtag, content)
        
        # Step 8: Replace emojis (remove them)
        content = emoji.replace_emoji(content, replace='')
        
        # Step 8.5: Replace non-ASCII apostrophes and quotes with standard versions
        # Apostrophes: ' (U+2019), ` (U+0060), ʼ (U+02BC), ´ (U+00B4), ' (U+2018)
        content = re.sub(r"[\u2019\u0060\u02BC\u00B4\u2018]", "'", content)
        
        # Quotes: " " (U+201C, U+201D), „ (U+201E), " (U+201F)
        content = re.sub(r"[\u201C\u201D\u201E\u201F]", '"', content)
        
        # Dashes/hyphens: — (em dash U+2014), – (en dash U+2013), ‐ (hyphen U+2010)
        content = re.sub(r"[\u2014\u2013\u2010]", "-", content)
        
        # Ellipsis: … (U+2026) and multiple dots
        content = re.sub(r"\u2026|\.{2,}", " ", content)
        
        # Remove quotes (both single and double)
        content = content.replace('"', '').replace("'", '')
        
        # Step 8.7: Remove standalone special symbols (bullets, arrows, etc.) - ONLY when standalone
        # These are only removed when they appear alone, not as part of a word
        for symbol in standalone_symbols:
            content = re.sub(rf'\s{re.escape(symbol)}\s', ' ', content)
            content = re.sub(rf'^{re.escape(symbol)}\s', '', content)
            content = re.sub(rf'\s{re.escape(symbol)}$', '', content)
        
        # Step 8.6: Handle contractions - expand them properly
        words = content.split()
        expanded_words = []
        
        for word in words:
            word_lower = word.lower()
            word_clean = re.sub(r'[^\w\s\'-]', '', word_lower)
            
            if word_clean in contractions_negative:
                expanded_words.append(contractions_negative[word_clean])
            elif word_clean in contractions_positive:
                expanded_words.append(contractions_positive[word_clean])
            else:
                expanded_words.append(word)
        
        content = ' '.join(expanded_words)
        
        # Step 9.1: Replace numbers with suffixes (7k, 213m, 5b, etc.) with "number"
        content = re.sub(r'\b\d+\.?\d*[kmbtKMBT]\b', 'number', content)
        # Step 9.2: Replace numbers with commas (e.g., 1,000, 1.000) with "number"
        content = re.sub(r'\b\d{1,3}(,\d{3})+(\.\d+)?\b', 'number', content)
        # Step 9.3: Replace ordered numbers (1st, 2nd, 3rd, 4th, etc.) with "number"
        content = re.sub(r'\b\d+(st|nd|rd|th)\b', 'number', content)
        
        # Step 9.4: Expand US state abbreviations (must be word boundaries)
        for abbr, full_name in us_states.items():
            # Use word boundaries to avoid matching parts of words
            content = re.sub(rf'\b{abbr}\b', full_name, content, flags=re.IGNORECASE)
        
        # Step 9.5: Expand country codes (prioritize 3-letter codes first to avoid conflicts)
        for code, full_name in sorted(countries.items(), key=lambda x: len(x[0]), reverse=True):
            content = re.sub(rf'\b{code}\b', full_name, content, flags=re.IGNORECASE)
        
        # Step 10: Process regular numbers
        def replace_number(match):
            num = match.group(0)
            num_type = classify_number(content, num)
            return num_type
        
        content = re.sub(r'\b\d+\b', replace_number, content)
        
        # Step 11: Normalize repeated characters
        content = normalize_repeated_chars(content)
        
        # Step 12: Translate internet slang/shorthand
        words = content.split()
        translated_words = []
        
        for word in words:
            word_lower = word.lower()
            # Remove punctuation for slang matching
            word_clean = re.sub(r'[^\w\s-]', '', word_lower)
            
            # Check if it's slang
            if word_clean in slang_map:
                translated_words.append(slang_map[word_clean])
            else:
                translated_words.append(word)
        
        content = ' '.join(translated_words)
        
        # Step 13: Handle remaining text
        words = content.split()
        cleaned_words = []
        
        for word in words:
            word_lower = word.lower()
            
            # Check for non-ASCII characters (after unidecode, this should be rare)
            if not all(ord(c) < 128 for c in word):
                has_non_ascii = True
                non_ascii_words.append(word)
                continue  # Skip non-ASCII words
            
            # Replace negative terms with "not"
            if word_lower in negative_terms:
                cleaned_words.append("not")
            # Remove common stop words
            elif word_lower not in remove_words:
                # Remove remaining special characters except hyphens in middle of words
                # This keeps "well-known" but removes "word." or ".word"
                cleaned_word = re.sub(r'^[^\w]+|[^\w]+$', '', word_lower)
                cleaned_word = re.sub(r'[^\w\s-]', '', cleaned_word)
                
                if cleaned_word and cleaned_word not in remove_words:
                    cleaned_words.append(cleaned_word)
        
        # Step 14: Join and normalize spaces
        content = ' '.join(cleaned_words)
        content = re.sub(r'\s+', ' ', content).strip()
        
        # Convert to lowercase
        content = content.lower()
        
        # Track corner cases
        corner_case_info = []
        if has_non_ascii:
            corner_case_info.append({
                "original": original_content,
                "non_ascii_words": non_ascii_words,
                "cleaned": content
            })
        
        return content, corner_case_info
    
    def extract_comment_content(comment, parent_author=None, depth=0):
        """Recursively extract and clean comment content"""
        username = comment.get("username")
        content = comment.get("content", "")
        
        # Clean the content
        cleaned_content, cases = clean_text(content, author=username, parent_author=parent_author)
        
        # Track corner cases
        corner_cases.extend(cases)
        
        # Add to list if not empty
        if cleaned_content:
            comment_lines.append(cleaned_content)
        
        # Process nested replies
        if "replies_content" in comment and comment["replies_content"]:
            for reply in comment["replies_content"]:
                extract_comment_content(reply, parent_author=username, depth=depth + 1)
    
    # Process each post
    for post in data:
        username = post.get("username")
        content = post.get("content", "")
        
        # Clean post content
        cleaned_content, cases = clean_text(content, author=username)
        
        # Track corner cases
        corner_cases.extend(cases)
        
        # Add to post lines if not empty
        if cleaned_content:
            post_lines.append(cleaned_content)
        
        # Process all comments for this post
        if "replies_content" in post and post["replies_content"]:
            for comment in post["replies_content"]:
                extract_comment_content(comment, parent_author=username, depth=0)
    
    # Save cleaned posts
    with open(posts_file, "w", encoding="utf-8") as f:
        for line in post_lines:
            f.write(line + "\n")
    
    # Save cleaned comments
    with open(comments_file, "w", encoding="utf-8") as f:
        for line in comment_lines:
            f.write(line + "\n")
    
    # Save corner cases
    if corner_cases:
        with open(corner_cases_file, "a", encoding="utf-8") as f:
            for case in corner_cases:
                f.write(f"Original: {case['original']}\n")
                f.write(f"Non-ASCII words: {', '.join(case['non_ascii_words'])}\n")
                f.write(f"Cleaned: {case['cleaned']}\n")
                f.write("-" * 80 + "\n")
    
    print(f"✅ Cleaned content saved:")
    print(f"   - Posts: {posts_file} ({len(post_lines)} lines)")
    print(f"   - Comments: {comments_file} ({len(comment_lines)} lines)")
    print(f"   - Corner cases: {corner_cases_file} ({len(corner_cases)} cases)")
    
    return {
        "posts_file": posts_file,
        "comments_file": comments_file,
        "posts_count": len(post_lines),
        "comments_count": len(comment_lines),
        "corner_cases_count": len(corner_cases)
    }

# Process all JSON files in a directory
def process_all_influencer_files_advanced(input_dir, output_dir, links_csv_path):
    """
    Process all JSON files with advanced cleaning.
    """
    input_dir = Path(input_dir)
    json_files = list(input_dir.glob("*.json"))
    
    print(f"Found {len(json_files)} JSON files to process\n")
    
    results = []
    for json_file in json_files:
        print(f"\n📄 Processing: {json_file.name}")
        try:
            result = clean_and_extract_content_advanced(json_file, output_dir, links_csv_path)
            results.append(result)
        except Exception as e:
            print(f"❌ Error processing {json_file.name}: {e}")
    
    # Summary
    total_posts = sum(r["posts_count"] for r in results)
    total_comments = sum(r["comments_count"] for r in results)
    total_corner_cases = sum(r["corner_cases_count"] for r in results)
    
    print(f"\n{'='*60}")
    print(f"📊 Processing Complete!")
    print(f"   - Files processed: {len(results)}/{len(json_files)}")
    print(f"   - Total posts: {total_posts}")
    print(f"   - Total comments: {total_comments}")
    print(f"   - Total corner cases: {total_corner_cases}")
    print(f"   - Output directory: {output_dir}")
    print(f"{'='*60}")
    
    return results

In [10]:
# Process single file
EXPR_DIR = "influencer_data/AdamParkhomenko.json"
OUTPUT_DIR = "outputs/cleaned_content_advanced"
LINKS_CSV_PATH = "outputs/links/links.csv"

# result = clean_and_extract_content_advanced(
#     EXPR_DIR, 
#     OUTPUT_DIR, 
#     LINKS_CSV_PATH
# )

# Process all files
results = process_all_influencer_files_advanced(
    "influencer_data",
    "outputs/cleaned_content_advanced",
    "outputs/links/links.csv"
)

Found 71 JSON files to process


📄 Processing: AdamParkhomenko.json
✅ Cleaned content saved:
   - Posts: outputs\cleaned_content_advanced\AdamParkhomenko_posts_cleaned.txt (40 lines)
   - Comments: outputs\cleaned_content_advanced\AdamParkhomenko_comments_cleaned.txt (1572 lines)
   - Corner cases: outputs\cleaned_content_advanced\corner_cases.txt (0 cases)

📄 Processing: ArielleScarcell.json
✅ Cleaned content saved:
   - Posts: outputs\cleaned_content_advanced\ArielleScarcell_posts_cleaned.txt (33 lines)
   - Comments: outputs\cleaned_content_advanced\ArielleScarcell_comments_cleaned.txt (2336 lines)
   - Corner cases: outputs\cleaned_content_advanced\corner_cases.txt (0 cases)

📄 Processing: AstroTerry.json
✅ Cleaned content saved:
   - Posts: outputs\cleaned_content_advanced\AstroTerry_posts_cleaned.txt (32 lines)
   - Comments: outputs\cleaned_content_advanced\AstroTerry_comments_cleaned.txt (1599 lines)
   - Corner cases: outputs\cleaned_content_advanced\corner_cases.txt (0 cases)